In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import numpy as np


# Example Dataset
class ExampleDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# Generate synthetic data
np.random.seed(42)
X = np.random.rand(1000, 10)  # 1000 samples, 10 features each
y = np.random.randint(0, 5, 1000)  # 5 classes (0 to 4)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create PyTorch datasets and dataloaders
train_dataset = ExampleDataset(X_train, y_train)
test_dataset = ExampleDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Define the Neural Network
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x


In [2]:
# Model, Loss, and Optimizer
input_size = X.shape[1]
num_classes = len(np.unique(y))
model = NeuralNetwork(input_size, num_classes)

criterion = nn.CrossEntropyLoss()  # For classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the class with highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")

Epoch 1/20, Loss: 1.6134
Epoch 2/20, Loss: 1.6092
Epoch 3/20, Loss: 1.6088
Epoch 4/20, Loss: 1.6060
Epoch 5/20, Loss: 1.6052
Epoch 6/20, Loss: 1.6039
Epoch 7/20, Loss: 1.6024
Epoch 8/20, Loss: 1.6001
Epoch 9/20, Loss: 1.5994
Epoch 10/20, Loss: 1.5985
Epoch 11/20, Loss: 1.5958
Epoch 12/20, Loss: 1.5933
Epoch 13/20, Loss: 1.5920
Epoch 14/20, Loss: 1.5890
Epoch 15/20, Loss: 1.5870
Epoch 16/20, Loss: 1.5849
Epoch 17/20, Loss: 1.5840
Epoch 18/20, Loss: 1.5801
Epoch 19/20, Loss: 1.5805
Epoch 20/20, Loss: 1.5779
Accuracy: 19.00%


In [8]:
inputs, labels = next(iter(train_loader))

In [12]:
inputs[0]

tensor([5.2746e-01, 1.4113e-01, 7.8951e-04, 8.1014e-01, 8.7575e-02, 6.3385e-01,
        9.1462e-01, 2.0865e-01, 6.6527e-01, 7.8699e-02])

In [13]:
labels[0]

tensor(1)

In [18]:
outputs = model(inputs)
outputs[0]

tensor([ 0.3822,  0.1151, -0.1845,  0.0443,  0.0594],
       grad_fn=<SelectBackward0>)

In [19]:
criterion(inputs[0], labels[0])

tensor(2.6205)